# Imports

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict,Counter
import re
import nltk
import pickle
import numpy as np
nltk.download('stopwords')

from nltk.corpus import stopwords
from tqdm import tqdm
import operator
from itertools import islice,count
from contextlib import closing

import json
from io import StringIO
from pathlib import Path
from operator import itemgetter
import pickle
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# TF-IDF Using TfidfVectorizer

In [51]:
def tf_idf_scores(data):
    """
    This function calculates the tfidf for each word in a single document utilizing TfidfVectorizer via sklearn.

    Parameters:
    -----------
      data: list of strings.

    Returns:
    --------
      Two objects as follows:
                                a) DataFrame, documents as rows (i.e., 0,1,2,3, etc'), terms as columns ('bird','bright', etc').
                                b) TfidfVectorizer object.

    """
    # YOUR CODE HERE
    vectorizer = TfidfVectorizer(stop_words='english')
    return pd.DataFrame(vectorizer.fit_transform(data).toarray(), columns=vectorizer.get_feature_names_out()), vectorizer

# Cosine Similarity Using Sklearn

In [54]:
def cosine_sim_using_sklearn(queries,tfidf):
    """
    In this function you need to utilize the cosine_similarity function from sklearn.
    You need to compute the similarity between the queries and the given documents.
    This function will return a DataFrame in the following shape: (# of queries, # of documents).
    Each value in the DataFrame will represent the cosine_similarity between given query and document.

    Parameters:
    -----------
      queries: sparse matrix represent the queries after transformation of tfidfvectorizer.
      documents: sparse matrix represent the documents.

    Returns:
    --------
      DataFrame: This function will return a DataFrame in the following shape: (# of queries, # of documents).
      Each value in the DataFrame will represent the cosine_similarity between given query and document.
    """
    # YOUR CODE HERE
    return pd.DataFrame(cosine_similarity(queries, tfidf))

# BM25

In [57]:
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stopwords_frozen = frozenset(stopwords.words('english'))
def tokenize(text):
    """
    This function aims in tokenize a text into a list of tokens. Moreover, it filters stopwords.

    Parameters:
    -----------
    text: string , repressing the text to tokenize.

    Returns:
    -----------
    list of tokens (e.g., list of tokens).
    """
    list_of_tokens =  [token.group() for token in RE_WORD.finditer(text.lower()) if token.group() not in stopwords_frozen]
    return list_of_tokens

[['sky', 'blue', 'see', 'blue', 'sun'],
 ['sun', 'bright', 'yellow'],
 ['comes', 'blue', 'sun'],
 ['lucy', 'sky', 'diamonds', 'see', 'sun', 'sky'],
 ['sun', 'sun', 'blue', 'sun', 'come'],
 ['lucy', 'likes', 'blue', 'bright', 'diamonds']]

In [ ]:
class BM25:
    """
    Best Match 25.

    Parameters to tune
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    Attributes
    ----------
    tf_ : list[dict[str, int]]
        Term Frequency per document, normalized by document's length.

    doc_len_ : list[int]
        Number of terms per document.

    df_ : dict[str, int]
        Document Frequency per term. i.e. Number of documents in the
        corpus that contains the term.

    avg_dl_ : float
        Average number of terms for documents in the corpus.

    idf_ : dict[str, float]
        Inverse Document Frequency per term.
    """

    def __init__(self, doc_len, df, tf=None, k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.tf_ = tf
        self.doc_len_ = doc_len
        self.df_ = df
        self.N_ = len(doc_len)
        self.avg_dl_ = sum(doc_len) / len(doc_len)
        self.idf_ = {}

    def calc_idf(self, query):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query.

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        # YOUR CODE HERE
        idf = {}

        for term in query:
            freq = 0
            if self.df_.get(term):
                freq = self.df_[term]
            idf[term] = math.log((self.N_ - freq + 0.5) / (freq + 0.5) + 1)

        return idf

    def search(self, queries):
        """
        This function use the _score function to calculate the bm25 score for all queries provided.

        Parameters:
        -----------
        queries: list of lists. Each inner list is a list of tokens.

        Returns:
        -----------
        list of scores of bm25
        """
        scores = []
        for query in queries:
            scores.append([self._score(query, doc_id) for doc_id in range(self.N_)])
        return scores

    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        score = 0.0

        if self.tf_:
            intersection = list(filter(lambda t: t in self.tf_[doc_id].keys(), query))
            b_calc = 1 - self.b + self.b * (self.doc_len_[doc_id] / self.avg_dl_)

            for term in intersection:
                score += ((self.k1 + 1) * self.tf_[doc_id][term]) / (
                            b_calc * self.k1 + self.tf_[doc_id][term]) * math.log2((self.N_ + 1) / self.df_[term])

        return score

In [64]:
def top_N_documents(df,N):
    """
    This function sort and filter the top N documents (by score) for each query.

    Parameters:
    ----------
    df: DataFrame (queries as rows, documents as columns)
    N: Integer (how many document to retrieve for each query)

    Returns:
    ----------
    top_N: dictionary is the following structure:
          key - query id.
          value - sorted (according to score) list of pairs length of N. Eac pair within the list provide the following information (doc id, score)
    """
    # YOUR CODE HERE
    result = {}

    for query_id, row in df.iterrows():
      doc_score = row.nlargest(N)
      result[query_id] = [(doc_id, score) for doc_id, score in doc_score.items()]

    return result

In [82]:
def generate_query_tfidf_vector(query_to_search, index):
    """
    Generate a vector representing the query. Each entry within this vector represents a tfidf score.
    The terms representing the query will be the unique terms in the index.

    We will use tfidf on the query as well.
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the query.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    Returns:
    -----------
    vectorized query with tfidf scores
    """
    epsilon = .0000001
    total_vocab_size = len(index.term_total)
    Q = np.zeros(total_vocab_size)
    term_vector = list(index.term_total.keys())
    counter = Counter(query_to_search)

    for token in np.unique(query_to_search):
        if token in index.term_total.keys(): #avoid terms that do not appear in the index.
            tf = counter[token] / len(query_to_search) # term frequency divded by the length of the query
            df = index.df[token]
            idf = math.log((len(index.dl)) / (df+epsilon), 10) #smoothing

            try:
                ind = term_vector.index(token)
                Q[ind] = tf*idf
            except:
                pass
    return Q

In [84]:
def get_candidate_documents_and_scores(query_to_search, index, words, pls):
    """
    Generate a dictionary representing a pool of candidate documents for a given query. This function will go through every token in query_to_search
    and fetch the corresponding information (e.g., term frequency, document frequency, etc.') needed to calculate TF-IDF from the posting list.
    Then it will populate the dictionary 'candidates.'
    For calculation of IDF, use log with base 10.
    tf will be normalized based on the length of the document.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.

    words,pls: iterator for working with posting.

    Returns:
    -----------
    dictionary of candidates. In the following format:
                                                               key: pair (doc_id,term)
                                                               value: tfidf score.
    """
    candidates = {}
    for term in np.unique(query_to_search):
        if term in words:
            list_of_doc = pls[words.index(term)]
            normalized_tfidf = [(doc_id, (freq / index.dl[str(doc_id)]) * math.log(len(index.dl) / index.df[term], 10)) for doc_id, freq in list_of_doc]

            for doc_id, tfidf in normalized_tfidf:
                candidates[(doc_id,term)] = candidates.get((doc_id,term), 0) + tfidf

    return candidates

In [85]:
def generate_document_tfidf_matrix(query_to_search,index,words,pls):
    """
    Generate a DataFrame `D` of tfidf scores for a given query.
    Rows will be the documents candidates for a given query
    Columns will be the unique terms in the index.
    The value for a given document and term will be its tfidf score.

    Parameters:
    -----------
    query_to_search: list of tokens (str). This list will be preprocessed in advance (e.g., lower case, filtering stopwords, etc.').
                     Example: 'Hello, I love information retrival' --->  ['hello','love','information','retrieval']

    index:           inverted index loaded from the corresponding files.


    words,pls: iterator for working with posting.

    Returns:
    -----------
    DataFrame of tfidf scores.
    """

    total_vocab_size = len(index.term_total)
    candidates_scores = get_candidate_documents_and_scores(query_to_search, index, words, pls) #No need to utilize all documents, only those having corresponding terms with the query.
    unique_candidates = np.unique([doc_id for doc_id, freq in candidates_scores.keys()])
    D = np.zeros((len(unique_candidates), total_vocab_size))
    D = pd.DataFrame(D)

    D.index = unique_candidates
    D.columns = index.term_total.keys()

    for key in candidates_scores:
        tfidf = candidates_scores[key]
        doc_id, term = key
        D.loc[doc_id][term] = tfidf

    return D

In [86]:
def cosine_similarity(D,Q):
    """
    Calculate the cosine similarity for each candidate document in D and a given query (e.g., Q).
    Generate a dictionary of cosine similarity scores
    key: doc_id
    value: cosine similarity score

    Parameters:
    -----------
    D: DataFrame of tfidf scores.

    Q: vectorized query with tfidf scores

    Returns:
    -----------
    dictionary of cosine similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: cosine similarty score.
    """
    # YOUR CODE HERE
    result = {}

    for doc_id, scores in D.iterrows():
      s = np.array(scores)
      result[doc_id] =np.dot(s,Q) / (np.linalg.norm(s) * np.linalg.norm(Q))

    return result

In [87]:
def get_top_n(sim_dict,N=3):
    """
    Sort and return the highest N documents according to the cosine similarity score.
    Generate a dictionary of cosine similarity scores

    Parameters:
    -----------
    sim_dict: a dictionary of similarity score as follows:
                                                                key: document id (e.g., doc_id)
                                                                value: similarity score. We keep up to 5 digits after the decimal point. (e.g., round(score,5))

    N: Integer (how many documents to retrieve). By default N = 3

    Returns:
    -----------
    a ranked list of pairs (doc_id, score) in the length of N.
    """

    return sorted([(doc_id,round(score,5)) for doc_id, score in sim_dict.items()], key = lambda x: x[1],reverse=True)[:N]

In [88]:
def get_topN_score_for_queries(queries_to_search,index,N=3):
    """
    Generate a dictionary that gathers for every query its topN score.

    Parameters:
    -----------
    queries_to_search: a dictionary of queries as follows:
                                                        key: query_id
                                                        value: list of tokens.
    index:           inverted index loaded from the corresponding files.
    N: Integer. How many documents to retrieve. This argument is passed to the topN function. By default N = 3, for the topN function.

    Returns:
    -----------
    return: a dictionary of queries and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id, score).
    """
    # YOUR CODE HERE
    result = {}

    for id, query in queries_to_search.items():
      words, pls = get_posting_iter(index)
      D = generate_document_tfidf_matrix(query, index, words, pls)
      Q = generate_query_tfidf_vector(query, index)
      result[id] = get_top_n(cosine_similarity(D,Q), N)

    return result

In [92]:
import math
from itertools import chain
import time
# When preprocessing the data have a dictionary of document length for each document saved in a variable called `DL`.
class BM25_from_index:
    """
    Best Match 25.
    ----------
    k1 : float, default 1.5

    b : float, default 0.75

    index: inverted index
    """

    def __init__(self,index,k1=1.5, b=0.75):
        self.b = b
        self.k1 = k1
        self.index = index
        self.N = len(DL)
        self.AVGDL = sum(DL.values())/self.N
        self.words, self.pls = zip(*self.index.posting_lists_iter())

    def calc_idf(self,list_of_tokens):
        """
        This function calculate the idf values according to the BM25 idf formula for each term in the query.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']

        Returns:
        -----------
        idf: dictionary of idf scores. As follows:
                                                    key: term
                                                    value: bm25 idf score
        """
        idf = {}
        for term in list_of_tokens:
            if term in self.index.df.keys():
                n_ti = self.index.df[term]
                idf[term] = math.log(1 + (self.N - n_ti + 0.5) / (n_ti + 0.5))
            else:
                pass
        return idf


    def search(self, queries,N=3):
        """
        This function calculate the bm25 score for given query and document.
        We need to check only documents which are 'candidates' for a given query.
        This function return a dictionary of scores as the following:
                                                                    key: query_id
                                                                    value: a ranked list of pairs (doc_id, score) in the length of N.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        # YOUR CODE HERE
        result = {}

        for query_id, query in queries.items():
          self.idf = self.calc_idf(query)
          candidates = set()

          for term in np.unique(query):
            if term in self.words:
              candidates.update(doc_freq[0] for doc_freq in self.pls[self.words.index(term)])

          result[query_id] = sorted([(doc_id, self._score(query, doc_id)) for doc_id in candidates], key=lambda x: x[1], reverse=True)[:min(N, self.N)]

        return result


    def _score(self, query, doc_id):
        """
        This function calculate the bm25 score for given query and document.

        Parameters:
        -----------
        query: list of token representing the query. For example: ['look', 'blue', 'sky']
        doc_id: integer, document id.

        Returns:
        -----------
        score: float, bm25 score.
        """
        score = 0.0
        doc_len = DL[str(doc_id)]

        for term in query:
            if term in self.index.term_total.keys():
                term_frequencies = dict(self.pls[self.words.index(term)])
                if doc_id in term_frequencies.keys():
                    freq = term_frequencies[doc_id]
                    numerator = self.idf[term] * freq * (self.k1 + 1)
                    denominator = freq + self.k1 * (1 - self.b + self.b * doc_len / self.AVGDL)
                    score += (numerator / denominator)
        return score

## Task 3: Using weights of title and body scores (25 points)

In [95]:
def merge_results(title_scores,body_scores,title_weight=0.5,text_weight=0.5,N = 3):
    """
    This function merge and sort documents retrieved by its weighte score (e.g., title and body).

    Parameters:
    -----------
    title_scores: a dictionary build upon the title index of queries and tuples representing scores as follows:
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)

    body_scores: a dictionary build upon the body/text index of queries and tuples representing scores as follows:
                                                                            key: query_id
                                                                            value: list of pairs in the following format:(doc_id,score)
    title_weight: float, for weigted average utilizing title and body scores
    text_weight: float, for weigted average utilizing title and body scores
    N: Integer. How many document to retrieve. This argument is passed to topN function. By default N = 3, for the topN function.

    Returns:
    -----------
    dictionary of querires and topN pairs as follows:
                                                        key: query_id
                                                        value: list of pairs in the following format:(doc_id,score).
    """
    # YOUR CODE HERE
    weighten_title_scores = {}
    for query_id in title_scores.keys():
      weighten_title_scores[query_id] = [(doc, score * title_weight) for doc, score in title_scores[query_id]]

    weighten_body_scores = {}
    for query_id in body_scores.keys():
      weighten_body_scores[query_id] = [(doc, score * text_weight) for doc, score in body_scores[query_id]]

    merged_scores = weighten_title_scores

    for query_id, docs_scores in weighten_body_scores.items():
      if merged_scores.get(query_id):
        merged_docs = {doc_id: score for doc_id, score in merged_scores[query_id]}

        for doc, score in docs_scores:
          if not merged_docs.get(doc):
            merged_docs[doc] = 0
          merged_docs[doc] += score

        merged_scores[query_id] = [(doc, score) for doc, score in merged_docs.items()]

      else:
        merged_scores[query_id] = weighten_body_scores[query_id]

      merged_scores[query_id] = sorted(merged_scores[query_id], key=lambda x: x[1], reverse=True)[:min(N, len(merged_scores[query_id]))]

    return merged_scores